In [2]:
! ls /home/ian.harry/tmp_abi

ls: cannot access '/home/ian.harry/tmp_abi': No such file or directory


In [3]:
!ls /home/ian.harry/tmp_abi/template_bank

ls: cannot access '/home/ian.harry/tmp_abi/template_bank': No such file or directory


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pylab as py
import pandas as pd 
import pycbc
import glob, os
import h5py
import lal 
import seaborn as sns

%matplotlib inline
#pycbc specifically for gravitational wave astronomy. For more information on github, follow this link: https://github.com/gwastro/pycbc
import pycbc.noise
import pycbc.filter
import pycbc.psd
from pycbc.filter import highpass_fir, matched_filter, matched_filter_core, MatchedFilterControl, sigmasq
from pycbc.filter.matchedfilter import Correlator
from pycbc.types import zeros
from pycbc.fft import IFFT
from pycbc.waveform import get_td_waveform, get_fd_waveform
from pycbc.waveform import td_approximants, fd_approximants

from scipy.signal import tukey


## Dictionaries 

In [ ]:
#Csv file of data ids and their coresponding targets. 
training_labels = pd.read_csv("/home/ian.harry/tmp_abi/training_labels.csv")
training_labels.head()

In [ ]:
#Dictionary containing ids and targets
tl = np.loadtxt('/home/ian.harry/tmp_abi/training_labels.csv',dtype='str', delimiter=',',skiprows=1)

dict_tl={}

for i in tl:
    dict_tl[i[0]] = i[1]  

In [ ]:
dict_tl['0000c3b9c9']

In [ ]:
keys_list = list(dict_tl)
a_key = keys_list[13]
print(a_key)

In [ ]:
print(keys_list[:10])
keys_list.index('0000c3b9c9')

In [ ]:
t_l = training_labels[:155]
tl_0 = t_l.loc[t_l['target'] == 0]
print(len(tl_0))

In [ ]:
len(data_in)

## PSD

In [ ]:
#Compute the PSD, code inspired from  my supervisor Ian Harry.
data_in = glob.glob('/home/ian.harry/tmp_abi/train/0/0/0/*.npy')

det1_psd = None 
det2_psd = None 
det3_psd = None 
count = 0

for filename in data_in:
    file_id = filename.split("/")[8].split('.')[0]
    
    if dict_tl[file_id] == 1:
        continue
    count += 1
    
    example_arr = np.load(filename)
    det1 = pycbc.types.TimeSeries(example_arr[0], delta_t= 1./2048.)#Splits the file path to just the id name
    det2 = pycbc.types.TimeSeries(example_arr[1], delta_t= 1./2048.) 
    det3 = pycbc.types.TimeSeries(example_arr[2], delta_t= 1./2048.)
    window = np.hanning(4096) 
    
    det1 = det1*window #applies window to minimise wrap arouind effect.
    det2 = det2*window
    det3 = det3*window 
    
    det1f = det1.to_frequencyseries() #Transform into the frequency domain
    det2f = det2.to_frequencyseries()
    det3f = det3.to_frequencyseries()
    
    if count == 1:
        det1_psd = abs(det1f)
        det2_psd = abs(det2f)
        det3_psd = abs(det3f)
    else:
        det1_psd = (abs(det1f) + (count-1) * det1_psd)/count
        det2_psd = (abs(det2f) + (count-1) * det2_psd)/count 
        det3_psd = (abs(det3f) + (count-1) * det3_psd)/count
        
det1_psd = det1_psd**2 * 4096/ (window*window).sum() # computes the average PSD
det2_psd = det2_psd**2 * 4096/ (window*window).sum()
det3_psd = det3_psd**2 * 4096/ (window*window).sum()
    

In [ ]:
%matplotlib inline
py.figure(figsize=(8,8))
py.loglog(det1_psd.sample_frequencies, det1_psd, label='Detector 1')
py.loglog(det2_psd.sample_frequencies, det2_psd, label='Detector 2')
py.loglog(det3_psd.sample_frequencies, det3_psd,label='Detector 3')
py.legend()
py.xlim(100,1100)
py.ylim(10e-50,10e-45)
py.xlabel('Frequency (Hz)')
py.ylabel('Strain')
PSD_match = plt.gcf()

In [ ]:
PSD_match.savefig('PSD_match.pdf',format='pdf')

## Generating Waveforms

In [ ]:
#Template bank provided by Ian Harry - contains 3000 waveforms
templates = h5py.File('/home/ian.harry/tmp_abi/template_bank/templates_097.h5', mode= 'r')
mass1s = (templates['mass1'][:1])
mass2s = (templates['mass2'][:1])
spin1z = (templates['spin1z'][:1])
spin2z = (templates['spin2z'][:1])

In [ ]:
#function to compute the lower frequency for the set of masses
def f_lower(M_chirp):
    return (5**(3/8))/(8*np.pi*((M_chirp**(5/3)*1)**(3/8)))

In [ ]:
#py.figure(figsize=(10,10))
for m1, m2, s1z, s2z in zip(mass1s,mass2s,spin1z,spin2z):
    mu = ((m1*m2)/(m1+m2))
    M = m1+m2
    M_chirp = (mu**(3/5))*(M**(2/5))*lal.MTSUN_SI # computing the chirp mass
    hp,hc = get_fd_waveform(approximant='IMRPhenomD',
                                                      mass1 = m1,
                                                      mass2 = m2, 
                                                      spin1z = s1z,
                                                      spin2z = s2z,
                                                      delta_f=det1_psd.delta_f, # frequency step
                                                      f_lower=f_lower(M_chirp),
                                                      f_upper=1024)
    hp.resize(len(det1_fs))
    #py.plot(hp.sample_frequencies,h )
print(hp.delta_f,len(hp))

In [ ]:
print(det1f.delta_f,hp.delta_f,det1_psd.delta_f)
hp.resize(len(det1f))

## Match Filtering

In [ ]:
#Assigning ids to a path
training_labels = pd.read_csv("/home/ian.harry/tmp_abi/training_labels.csv")

path = list(training_labels['id'])
for i in range(len(path)):
    path[i] = '/home/ian.harry/tmp_abi/train/' + path[i][0] +  '/' + path[i][1] +  '/' + path[i][2] +  '/' + path[i] + '.npy'
print(path[1])

In [ ]:
data_size = len(path)//50 # reducing the data size
print(data_size)
data_in = path[0:data_size]

In [ ]:
#Same principle as my code but optimized by Ian Harry

det_filter_final = []
from pycbc.filter.matchedfilter import _BaseCorrelator

class CPUCorrelator(_BaseCorrelator):
    def __init__(self, x, y, z):
        self.x = np.array(x.data, copy=False)
        self.y = np.array(y.data, copy=False)
        self.z = np.array(z.data, copy=False)

    def correlate(self):
        self.z[:] = np.conjugate(self.x.data)
        self.z *= self.y

print("Pregenerating waveforms and normalizations")
pregenerated_waveforms = []
pregenerated_sigma1s = []
pregenerated_sigma2s = []
pregenerated_sigma3s = []

for m1, m2, s1z, s2z in zip(mass1s,mass2s,spin1z,spin2z):
        mu = ((m1*m2)/(m1+m2))
        M = m1+m2
        M_chirp = (mu**(3/5))*(M**(2/5))*lal.MTSUN_SI
        hp,hc = get_fd_waveform(approximant='IMRPhenomD',
                                                          mass1 = m1,
                                                          mass2 = m2, 
                                                          spin1z = s1z,
                                                          spin2z = s2z,
                                                          delta_f= det1_psd.delta_f,
                                                          f_lower=f_lower(M_chirp),
                                                          f_upper=1000)
        hp.resize(len(det1_psd))
        pregenerated_waveforms.append(hp)
        # sigma squared is a normalisation constant to compute the correct SNR.
        template_norm1 = sigmasq(hp, det1_psd, low_frequency_cutoff=30, high_frequency_cutoff=1000)
        template_norm2 = sigmasq(hp, det2_psd, low_frequency_cutoff=30, high_frequency_cutoff=1000)
        template_norm3 = sigmasq(hp, det3_psd, low_frequency_cutoff=30, high_frequency_cutoff=1000)
        pregenerated_sigma1s.append(template_norm1)
        pregenerated_sigma2s.append(template_norm2)
        pregenerated_sigma3s.append(template_norm3)

print("Beginning to process data")

    
det_filter_final = []
for filname in data_in:
    
    ##### GET DATA
    ex_arr = np.load(filname)
    print(filname, filname.split('/')[-1][:-4], dict_tl[filname.split('/')[-1][:-4]])

    ex_arr = np.load(filname)
    window = tukey(4096, alpha=0.05)

    det1_ts = pycbc.types.TimeSeries(ex_arr[0], delta_t= 1./2048.)
    det1_ts = det1_ts * window # I think you need a window here!
    det2_ts = pycbc.types.TimeSeries(ex_arr[1], delta_t= 1./2048.) 
    det2_ts = det2_ts * window # I think you need a window here!
    det3_ts = pycbc.types.TimeSeries(ex_arr[2], delta_t= 1./2048.)
    det3_ts = det3_ts * window # I think you need a window here!


    det1_fs = det1_ts.to_frequencyseries()
    det2_fs = det2_ts.to_frequencyseries()
    det3_fs = det3_ts.to_frequencyseries()
    
    # Apply PSD here
    det1_fs = det1_fs / det1_psd
    det2_fs = det2_fs / det2_psd
    det3_fs = det3_fs / det3_psd
    
    ##### SET UP CORRELATORS AND FFTs

    tlen = 2 * 2048 # Length of data is 4096 data points
    flen = tlen // 2 + 1
    data_time_length = 2
    template_mem1 = zeros(tlen, dtype = np.complex128)
    template_mem2 = zeros(tlen, dtype = np.complex128)
    template_mem3 = zeros(tlen, dtype = np.complex128)
    snr_mem1 = zeros(tlen, dtype=np.complex128)
    snr_mem2 = zeros(tlen, dtype=np.complex128)
    snr_mem3 = zeros(tlen, dtype=np.complex128)
    corr_mem1 = zeros(tlen, dtype=np.complex128)
    corr_mem2 = zeros(tlen, dtype=np.complex128)
    corr_mem3 = zeros(tlen, dtype=np.complex128)
    # Low and high frequency cutoffs
    kmin = 30 * data_time_length
    kmax = 1000 * data_time_length
    corr_slice = slice(kmin, kmax)
    corr1 = CPUCorrelator(template_mem1[corr_slice], det1_fs[corr_slice], corr_mem1[corr_slice])
    corr2 = CPUCorrelator(template_mem2[corr_slice], det2_fs[corr_slice], corr_mem2[corr_slice])
    corr3 = CPUCorrelator(template_mem3[corr_slice], det3_fs[corr_slice], corr_mem3[corr_slice])
    ifft1 = IFFT(corr_mem1, snr_mem1)
    ifft2 = IFFT(corr_mem2, snr_mem2)
    ifft3 = IFFT(corr_mem3, snr_mem3)
        
    max_icv = 0
    count = 0
    for (hp, ps1, ps2, ps3) in zip(pregenerated_waveforms, pregenerated_sigma1s, pregenerated_sigma2s, pregenerated_sigma3s):

        count += 1
        if count % 1000 == 0:
            print (count, 'of', len(mass1s)) # basic tqdm
        template_mem1._data[:len(hp)] = hp._data[:]
        template_mem2._data[:len(hp)] = hp._data[:]
        template_mem3._data[:len(hp)] = hp._data[:]
        
        template_norm1 = ps1
        template_norm2 = ps2
        template_norm3 = ps3

        norm1 = (4.0 * det1_psd.delta_f) / np.sqrt(template_norm1)
        norm2 = (4.0 * det2_psd.delta_f) / np.sqrt(template_norm2)
        norm3 = (4.0 * det3_psd.delta_f) / np.sqrt(template_norm3)
        
        corr1.correlate()
        ifft1.execute()
        
        corr2.correlate()
        ifft2.execute()

        corr3.correlate()
        ifft3.execute()
                 
        # NOTE [2048:3686] is -1, -0.2 ... But I swapped to 3030:3745
        max_det1_filter = abs(snr_mem1._data[3030:3745]).argmax()
        det1_max = abs(snr_mem1._data[max_det1_filter+3030]) * norm1 # finding the max snr
    
        max_det2_filter = abs(snr_mem2._data[3030:3745]).argmax()
        det2_max = abs(snr_mem2._data[max_det2_filter+3030]) * norm2
  
        max_det3_filter = abs(snr_mem3._data[3030:3745]).argmax()
        det3_max = abs(snr_mem3._data[max_det3_filter+3030]) * norm3
        
        max_filter = (det1_max+det2_max+det3_max)
        
        
        if max_filter > max_icv:
            max_icv = max_filter
            
                
    
    det_filter_final.append((max_icv, filname.split('/')[-1][:-4]))
  



In [ ]:
print(len(det_filter_final))


In [ ]:
# Writing the Data
import csv
with open('filterfile.txt',"w") as f:
    csv.writer(f, delimiter =',').writerows(det_filter_final) # Writing the list to a file 

In [ ]:
#Reading in the data
final_dat = pd.read_csv('filterfile.txt',sep=',', header = None)
final_data = final_dat.iloc[:,0]
print(len(final_data))

In [ ]:
dat = pd.DataFrame(list())
dat.to_csv('matchv1.csv')
matchv1 = pd.read_csv('matchv1.csv') # Creating a csv file 

In [ ]:
ids = training_labels['id'].values[0:data_size]
actual_targets = training_labels['target'].values[0:data_size]
print(len(ids),len(actual_targets))

In [ ]:
match_target = np.array(final_data)
match_target[:,] = match_target[:,] > 15 # Setting a threshold for binary classification 
match_target = match_target.astype(int)
print(match_target)
print(len(match_target))

In [ ]:
frac_target = []
mat_tar = list(match_target)
actu_tar = list(actual_targets)

for i in range(len(mat_tar)):
    if mat_tar[i] == actu_tar[i]:
        frac_target.append(1)
    else:
        frac_target.append(0)
print(len(frac_target))

In [ ]:
matchv1 = pd.DataFrame({'id':ids, 'Max SNRs': final_data, 
                     'Match Targets': match_target,
                     'Actual Targets': actual_targets,
                     'Classified Targets': frac_target})
matchv1.to_csv('matchv1.csv',index = False)

In [ ]:
matchv1.head(10)

In [ ]:
no_ones = np.count_nonzero(frac_target)
acc_score_match = (no_ones/len(frac_target))*100
error_score_match = (100-acc_score_match)
print('Classified Correctly: ',acc_score_match, 'Classified Incorrectly: ',error_score_match)